In [1]:
import numpy as np
%load_ext autoreload
%autoreload 2
from gridworld_utils import print_grid_values, print_grid_policy

# Policy Evaluation

Finding the state-values for a given policy in some gridworld where 
* There are 4 actions; left, up, right, down. (If you move into the edge you stay where you are.)
* You get a reward of -1 for each step
* Terminal states have a value of 0

The first example is a 4x4 gridworld with two corner terminal states, like the example in chapter 4.1 of Sutton and Barto's 'Rienforcement Learning: An Introduction'.

In [7]:
def update_state_value(policy, state_values, reward=-1, discount=1):
    v = 0
    # Bellman Expectation Equation for deterministic environment. (Taking
    # some action in some state will always lead to the same successor state).
    for p, s_v in zip(policy, state_values):
        v += p * (reward + discount * s_v)
    return v

In [58]:
def one_sweep_policy_evaluation(grid, policy, terminal_states):
    rows, columns = grid.shape
    new_grid = np.zeros(grid.shape)
    for i in range(rows):
        for j in range(columns):
            
            # Making sure not to update terminal states
            is_terminal_state = False
            for terminal_state in terminal_states:
                x, y = terminal_state
                if i == x and j == y:
                    is_terminal_state = True
            if is_terminal_state:
                continue
                
            # Finding previous state values for all possible succussor states 
            v_left = grid[i][j] if j == 0 else grid[i][j-1]
            v_up = grid[i][j] if i == 0 else grid[i-1][j]
            v_right = grid[i][j] if j == (columns-1) else grid[i][j+1]
            v_down = grid[i][j] if i == (rows-1) else grid[i+1][j]
            
            new_grid[i][j] = update_state_value(policy[i][j], (v_left,v_up,v_right,v_down))
    return new_grid
    

In [85]:
def full_policy_evaluation(grid, policy, terminal_states, theta=0.01):
    difference = 100 # Arbitrary large initial difference 
    iteration = 0 
    
    while difference > theta:
        previous_grid = np.copy(grid)
        grid = one_sweep_policy_evaluation(grid, policy, terminal_states)
        difference = np.max(np.abs(previous_grid - grid))
        iteration += 1
    return grid, iteration

In [37]:
def initialise_random_policy(shape, terminal_states):
    policy = np.empty(shape, dtype=object)
    policy.fill((1/4, 1/4, 1/4, 1/4))
    for terminal_state in terminal_states:
        policy[terminal_state] = None
    return policy

### Examples

In [68]:
gridworld_4x4 = np.zeros((4,4))
terminal_states = ((0,0), (3,3)) # Top left and bottom right corners
policy = initialise_random_policy(gridworld_4x4.shape, terminal_states)

for k in range(4):
    print('State-values at iteration {}:'.format(k))
    print_grid_values(gridworld_4x4)
    gridworld_4x4 = one_sweep_policy_evaluation(gridworld_4x4, policy, terminal_states)
    

State-values at iteration 0:
|=======|=======|=======|=======|
|  0.0  |  0.0  |  0.0  |  0.0  |
|=======|=======|=======|=======|
|  0.0  |  0.0  |  0.0  |  0.0  |
|=======|=======|=======|=======|
|  0.0  |  0.0  |  0.0  |  0.0  |
|=======|=======|=======|=======|
|  0.0  |  0.0  |  0.0  |  0.0  |
|=======|=======|=======|=======|

State-values at iteration 1:
|=======|=======|=======|=======|
|  0.0  | -1.0  | -1.0  | -1.0  |
|=======|=======|=======|=======|
| -1.0  | -1.0  | -1.0  | -1.0  |
|=======|=======|=======|=======|
| -1.0  | -1.0  | -1.0  | -1.0  |
|=======|=======|=======|=======|
| -1.0  | -1.0  | -1.0  |  0.0  |
|=======|=======|=======|=======|

State-values at iteration 2:
|=======|=======|=======|=======|
|  0.0  | -1.8  | -2.0  | -2.0  |
|=======|=======|=======|=======|
| -1.8  | -2.0  | -2.0  | -2.0  |
|=======|=======|=======|=======|
| -2.0  | -2.0  | -2.0  | -1.8  |
|=======|=======|=======|=======|
| -2.0  | -2.0  | -1.8  |  0.0  |
|=======|=======|=======|==

In [89]:
gridworld_6x6 = np.zeros((6,6))
terminal_states = ((2,2), (2,3), (3,2), (3,3)) # The middle 4 squares
policy = initialise_random_policy(gridworld_6x6.shape, terminal_states)
theta = 0.001

gridworld_6x6, k = full_policy_evaluation(gridworld_6x6, policy, terminal_states, theta)
print('State-values after iteration {}:'.format(k))
print_grid_values(gridworld_6x6)


State-values after iteration 159:
|=======|=======|=======|=======|=======|=======|
| -27.0 | -25.0 | -22.5 | -22.5 | -25.0 | -27.0 |
|=======|=======|=======|=======|=======|=======|
| -25.0 | -21.5 | -16.0 | -16.0 | -21.5 | -25.0 |
|=======|=======|=======|=======|=======|=======|
| -22.5 | -16.0 |  0.0  |  0.0  | -16.0 | -22.5 |
|=======|=======|=======|=======|=======|=======|
| -22.5 | -16.0 |  0.0  |  0.0  | -16.0 | -22.5 |
|=======|=======|=======|=======|=======|=======|
| -25.0 | -21.5 | -16.0 | -16.0 | -21.5 | -25.0 |
|=======|=======|=======|=======|=======|=======|
| -27.0 | -25.0 | -22.5 | -22.5 | -25.0 | -27.0 |
|=======|=======|=======|=======|=======|=======|



# Policy Iteration

Same gridworld rules as stated above.

Whats happeing:
1. Beginning with a random policy and all state-values set to 0
2. Evaluating that policy all the way untill changes being made are less than set parameter theta (**Policy Evaluation**)
3. Acting greedily on the values for that policy to generate a better policy (**Policy Improvement**)
4. Repeating steps 2 and 3 untill the improved policies are equal to each other and thus equal to the optimal policy (**Policy Iteration**)

In [34]:
# Getting an improved Policy by taking greedy actions with respect to the
# immediate reward + value of next state
def get_improved_policy(grid, terminal_states, reward=-1, discount=1):
    rows, columns = grid.shape
    policy = np.empty(grid.shape, dtype=object)
    for i in range(rows):
        for j in range(columns):
            
            # Making sure not to update terminal states
            is_terminal_state = False
            for terminal_state in terminal_states:
                x, y = terminal_state
                if i == x and j == y:
                    is_terminal_state = True
            if is_terminal_state:
                continue
                
            # Finding previous state values for all possible succussor states 
            v_left = grid[i][j] if j == 0 else grid[i][j-1]
            v_up = grid[i][j] if i == 0 else grid[i-1][j]
            v_right = grid[i][j] if j == (columns-1) else grid[i][j+1]
            v_down = grid[i][j] if i == (rows-1) else grid[i+1][j]
            
            # Finding the the values of every action from current state and
            # acting greedly 
            successor_state_values = np.array([v_left, v_up, v_right, v_down])
            action_values = reward + discount * successor_state_values
            greedy_actions = action_values == np.max(action_values)
            policy[i][j] = tuple(greedy_actions / np.count_nonzero(greedy_actions))
    return policy
            

In [128]:
def policy_iteration(grid, policy, terminal_states, theta):
    print('Initial state-values and policy:')
    print_grid_values(grid)
    print_grid_policy(policy)

    improved_policies = 0
    total_k = 0
    while True:
        grid, k = full_policy_evaluation(grid, policy, terminal_states, theta)
        total_k += k
        
        previous_policy = np.copy(policy)
        policy = get_improved_policy(grid, terminal_states)
        improved_policies += 1
        
        print('State-values and improved policy #{} (k={}):'.format(improved_policies, total_k))
        print_grid_values(grid)
        print_grid_policy(policy)
        
        if (previous_policy == policy).all():
            print('Optimal policy found!')
            return grid, policy

### Examples

In [126]:
gridworld_4x4 = np.zeros((4,4))
terminal_states = ((0,0), (3,3)) # Top left and bottom right corners
policy = initialise_random_policy(gridworld_4x4.shape, terminal_states)

print('Initial state-values and policy:')
print_grid_values(gridworld_4x4)
print_grid_policy(policy)

for k in range(4):
    gridworld_4x4 = one_sweep_policy_evaluation(gridworld_4x4, policy, terminal_states)
print('State-values and greedy policy w.r.t. v after 3 iterations:')
print_grid_values(gridworld_4x4)
improved_policy = get_improved_policy(gridworld_4x4, terminal_states)
print_grid_policy(improved_policy)

Initial state-values and policy:
|=======|=======|=======|=======|
|  0.0  |  0.0  |  0.0  |  0.0  |
|=======|=======|=======|=======|
|  0.0  |  0.0  |  0.0  |  0.0  |
|=======|=======|=======|=======|
|  0.0  |  0.0  |  0.0  |  0.0  |
|=======|=======|=======|=======|
|  0.0  |  0.0  |  0.0  |  0.0  |
|=======|=======|=======|=======|

|=======|=======|=======|=======|
|///////| ←↑→↓  | ←↑→↓  | ←↑→↓  |
|=======|=======|=======|=======|
| ←↑→↓  | ←↑→↓  | ←↑→↓  | ←↑→↓  |
|=======|=======|=======|=======|
| ←↑→↓  | ←↑→↓  | ←↑→↓  | ←↑→↓  |
|=======|=======|=======|=======|
| ←↑→↓  | ←↑→↓  | ←↑→↓  |///////|
|=======|=======|=======|=======|

State-values and greedy policy w.r.t. v after 3 iterations:
|=======|=======|=======|=======|
|  0.0  | -3.1  | -3.8  | -4.0  |
|=======|=======|=======|=======|
| -3.1  | -3.7  | -3.9  | -3.8  |
|=======|=======|=======|=======|
| -3.8  | -3.9  | -3.7  | -3.1  |
|=======|=======|=======|=======|
| -4.0  | -3.8  | -3.1  |  0.0  |
|=======|=======|====

In [129]:
gridworld_8x8 = np.zeros((8,8))
terminal_states = ((2, 2), (3, 2), (4, 2), (4, 3), (6, 6), (7, 0))
policy = initialise_random_policy(gridworld_8x8.shape, terminal_states)
theta = 0.01

v, pi = policy_iteration(gridworld_8x8, policy, terminal_states, theta)

Initial state-values and policy:
|=======|=======|=======|=======|=======|=======|=======|=======|
|  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |
|=======|=======|=======|=======|=======|=======|=======|=======|
|  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |
|=======|=======|=======|=======|=======|=======|=======|=======|
|  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |
|=======|=======|=======|=======|=======|=======|=======|=======|
|  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |
|=======|=======|=======|=======|=======|=======|=======|=======|
|  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |
|=======|=======|=======|=======|=======|=======|=======|=======|
|  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |
|=======|=======|=======|=======|=======|=======|=======|=======|
|  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |  0.0  |
|=======|=======|=======|=======|=======|==

**Change the hyperparameters above to create your own gridworld and then find the optimal value function and policy** 

# Value Iteration